In [1]:
import torch
import torch.nn as nn
from Model import model, modelArguments
import time
import re
import math
import triton

In [2]:
fmodel = model(modelArguments)

state_dict = torch.load('./checkpoints/model_weightsBPE6.pth')
new_state_dict = {k.replace('_orig_mod.', ''): v for k, v in state_dict.items()}
fmodel.load_state_dict(new_state_dict, strict=False)

fmodel = fmodel.to('cuda')

/tmp/ipykernel_47271/3021890348.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./checkpoints/model_weightsBPE6.pth')


In [3]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="large_tokenizer.json")


def tokenize(text, second = False):
    ids = tokenizer.encode(text, add_special_tokens=True)
    ids1 = [1] + ids
    ids1 = torch.tensor(ids1, dtype=torch.int16)
    
    # # Pad or truncate the sequence to tholtae max_seq_length
    # if ids1.size(0) < modelArguments.max_seq_len:
    #     padding_length = modelArguments.max_seq_len - ids1.size(0)
    #     ids1 = torch.cat([ids1, torch.zeros(padding_length, dtype=torch.int16)])
        
    # elif ids1.size(0) >= modelArguments.max_seq_len:
    #     ids1 = ids1[:modelArguments.max_seq_len]
    #     ids1[-1] = 2
    return ids1

/home/frosthead/.conda/envs/DL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def id_to_token(x):
    s = ''
    for i in x[0]:
        # j = torch.argmax(i)
        s += tokenizer.decode(i)

    return s

def id_to_token_M(x):
    s = ''
    for i in x[0]:
        j = torch.argmax(i)
        s += tokenizer.decode(j)

    return s

In [5]:
fmodel.eval()

model(
  (shared_emb): shared_emb()
  (enc_layers): ModuleList(
    (0-9): 10 x EncoderBlock(
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (attention): FlashAttention(
        (q_linear): Linear(in_features=512, out_features=512, bias=False)
        (k_linear): Linear(in_features=512, out_features=512, bias=False)
        (v_linear): Linear(in_features=512, out_features=512, bias=False)
        (out_linear): Linear(in_features=512, out_features=512, bias=False)
      )
      (feedforward): FeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=False)
        (fc2): Linear(in_features=2048, out_features=512, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
)

In [6]:
# test sentence

sentence = """MY name is Ayush People also call me?"""
generated_id  = tokenize(sentence).to(torch.long).to('cuda').unsqueeze(0)
generated_ids = torch.cat([generated_id for i in range(modelArguments.max_batch_size)], dim=0)

print(generated_ids.shape)
print(generated_ids)

torch.Size([8, 12])
tensor([[    1,    48,    60,  2370,   310, 15529,  2346,  6241,   830,  1928,
           527,    34],
        [    1,    48,    60,  2370,   310, 15529,  2346,  6241,   830,  1928,
           527,    34],
        [    1,    48,    60,  2370,   310, 15529,  2346,  6241,   830,  1928,
           527,    34],
        [    1,    48,    60,  2370,   310, 15529,  2346,  6241,   830,  1928,
           527,    34],
        [    1,    48,    60,  2370,   310, 15529,  2346,  6241,   830,  1928,
           527,    34],
        [    1,    48,    60,  2370,   310, 15529,  2346,  6241,   830,  1928,
           527,    34],
        [    1,    48,    60,  2370,   310, 15529,  2346,  6241,   830,  1928,
           527,    34],
        [    1,    48,    60,  2370,   310, 15529,  2346,  6241,   830,  1928,
           527,    34]], device='cuda:0')


In [7]:
t0 = time.time()

for _ in range(modelArguments.max_seq_len-generated_ids.size(1)):  # Generate up to 20 tokens
    with torch.no_grad():
        logits = fmodel(generated_ids)  # Pass the current sequence to the model
        next_token_logits = logits[:, -1, :]  # Get logits for the last token


    # Predict the next token (greedy decoding)
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(1)  # Shape: [1, 1]

    # Append the predicted token to the sequence
    generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

    # Stop if EOS token is predicted
    # if next_token_id.item() == 2:
    #     print("eos")
    #     break

t1 = time.time()
# Decode the generated sequence
output_text = tokenizer.decode(generated_ids[0])
print("Generated text:", output_text)
t = (t1-t0)*1000
print("time taken ", t)


torch.Size([12, 512])
torch.Size([8, 12, 512])
torch.Size([13, 512])
torch.Size([8, 13, 512])
torch.Size([14, 512])
torch.Size([8, 14, 512])
torch.Size([15, 512])
torch.Size([8, 15, 512])
torch.Size([16, 512])
torch.Size([8, 16, 512])
torch.Size([17, 512])
torch.Size([8, 17, 512])
torch.Size([18, 512])
torch.Size([8, 18, 512])
torch.Size([19, 512])
torch.Size([8, 19, 512])
torch.Size([20, 512])
torch.Size([8, 20, 512])
torch.Size([21, 512])
torch.Size([8, 21, 512])
torch.Size([22, 512])
torch.Size([8, 22, 512])
torch.Size([23, 512])
torch.Size([8, 23, 512])
torch.Size([24, 512])
torch.Size([8, 24, 512])
torch.Size([25, 512])
torch.Size([8, 25, 512])
torch.Size([26, 512])
torch.Size([8, 26, 512])
torch.Size([27, 512])
torch.Size([8, 27, 512])
torch.Size([28, 512])
torch.Size([8, 28, 512])
torch.Size([29, 512])
torch.Size([8, 29, 512])
torch.Size([30, 512])
torch.Size([8, 30, 512])
torch.Size([31, 512])
torch.Size([8, 31, 512])
torch.Size([32, 512])
torch.Size([8, 32, 512])
torch.Size([3

In [8]:
generated_ids.shape

torch.Size([8, 256])

In [9]:
next_token_id.shape

torch.Size([8, 1])

In [10]:
output_text = tokenizer.decode(generated_ids[0])
print("Generated text:", output_text)

Generated text: [SOS]MY name is Ayush People also call me?
The most to the most way to the most to the best of the the most way to the best of the most to the new and the the time of the most time of the first--based and the most to be a new and the best of the best of the best of the most than the best of the most to the most than the most more to the best of the most to the most way to the most time, you can be a new time, you to be a own of the most to be a a own time to be a own to be a new--s the time, you to be a own of the best of the best to be a new-s, you’s a time, you’s a a time, you to be a own of the best, you to the time to be a new-based, you can be a a own of the the the time to the time, you to be a time to be a time, you’s a time to be to be to be to the time to the time, you’s a a time to be a new time to be a own of the the time, you to be a new time of


In [11]:
output_text = tokenizer.decode(generated_ids[1])
print("Generated text:", output_text)

Generated text: [SOS]MY name is Ayush People also call me?
The first-----
-
--
--
-----------
-
-
-
-
-
-
-
-
-
-
-
-
--
--
-
--
-
-
-
-
-
-
-
-
--
--
-
-
-
---
-
-----
--
---
--
-
-
-
-
-
--
-
---
-
-
------
--------
-
-
---
---
---
-
--
-
--
-
--
--
-----
--
----
------
-
----------
-------
---
-
----
-
-



In [12]:
output_text = tokenizer.decode(generated_ids[2])
print("Generated text:", output_text)

Generated text: [SOS]MY name is Ayush People also call me?
The first and the most is a new and the most to the first of the first, the most is a own to the best of the most to the best of the best of the most more to be a new of the best to the best of the most to be a own time, and the best of the most to the most than a new and the first-based is a time, and the best, and the best of the best of the best of the time, and the most to be a own of the best to be a new and the most to the most than a own of the the most way to the time of the best of the best of the the time of the best of the best of the best of the best of the world of the-----based and the most to the first of the best of the best of the most to the most is a new, and a time of the most way to the time, and the time, and the the most than a time, and the best of the best of the time of the time to the most to the time to be a time, you’s a new--s to the time, and you[EOS]
